In [ ]:
# IMPORTS ALL THE REQUIRED MODULES

from pymatgen.entries.compatibility import MaterialsProject2020Compatibility

mpc = MaterialsProject2020Compatibility(check_potcar=False)

from mp_api.client import MPRester
from emmet.core.thermo import ThermoType
import pymatgen
import csv

mpr = MPRester("PGh5dJz2L9dILgfgixXM8TH0cK14ri5m")  # NEED TO SET API KEY

from pymatgen.analysis.phase_diagram import PDEntry, PhaseDiagram
from pymatgen.entries.computed_entries import ComputedEntry

from ase import Atoms
from ase.io import read

import numpy as np
import time

In [ ]:
# IMPORTS THE REQUIRED THERMO DOCS FOR CONSTRUCTING THE HULL

thermo_docs = []
with open('thermo docs.csv', mode='r', newline='') as file:    # CHANGE IMPORT FILE HERE
    reader = csv.reader(file)
    for row in reader: 
        thermo_docs.append(row)  # Convert each value


In [ ]:
# CHANGE THE DATA TYPE OF THE IMPORTED DATA

for i in thermo_docs:
    i[0] = pymatgen.core.composition.Composition(i[0])
    i[1] = float(i[1])


In [ ]:
# CHECK THAT THE DATA TYPE HAS BEEN MODIFIED SUCCESSFULLY

print(type(thermo_docs[2][1]))
print(type(thermo_docs[2][0]))
print(len(thermo_docs))

In [ ]:
# CHANGE HERE, CHANGE INFO FILE NAME AND CHANGE INDEX IN E HULL FUNCTION

phase = "Phase 2 2.0"

In [ ]:
import os

folder_path = f"{phase}"  # CHANGE DIRECTORY

file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
len(file_list)


In [ ]:
file_list_2 = [f for f in os.listdir("Phase 3 2.0") if os.path.isfile(os.path.join("Phase 3 2.0", f))]
print(len(file_list_2))

In [ ]:
phase_info = []
with open('Phase 2 2.0 4072.csv', mode='r', newline='') as file:    # CHANGE IMPORT FILE HERE
    reader = csv.reader(file)
    for row in reader: 
        phase_info.append(row)  # Convert each value

print(len(phase_info))

In [ ]:
phase_info[0]

In [ ]:
phase_full_info = []
for c,i in enumerate(phase_info[1:]):
    phase_full_info.append([c+5482,i[:3], i[3:6], i[9], i[10], i[11],(float(i[9])+float(i[11]))/(np.sqrt(2)*(float(i[10])+float(i[11]))),float(i[10])/float(i[11])])

print(len(phase_full_info))
print(phase_full_info[0])

In [ ]:
def energy_above_hull_3(file_list,thermo_docs):
    
    start = time.process_time()
    start_2 = time.time()
    count = 0
    fail = []
    f_count = 0
    fail_count = 0
    result_list = []
    
    for c,i in enumerate(file_list):
        try:
            count += 1
            atoms = read(f"{phase}/{i}")     # CHANGE TO APPROPRATE FOLDER NAME
            compound_symbol = atoms.symbols
            individual_symbol = atoms.get_chemical_symbols()
            free_energy = atoms.get_potential_energy()
    
            # Energy above hull data
            
            chemsys = {f"{individual_symbol[0]}",f"{individual_symbol[2]}",f"{individual_symbol[3]}"}
            
            entries = []
            for doc in thermo_docs:
                elements = {e.name for e in doc[0].elements}
                if elements.issubset(chemsys):
                    entry = PDEntry(
                        doc[0],
                        doc[1] * doc[0].num_atoms
                        )
                    entries.append(entry)
            #print(entries)
    
            pd = PhaseDiagram(entries)
    
            entry = ComputedEntry(
                f"{compound_symbol}",
                free_energy,
                parameters={"run_type": "GGA+U"}
                )
            
            corrected_entry = mpc.process_entry(entry)
            #print(corrected_entry)
            a = pd.get_e_above_hull(corrected_entry)
            #print(a)
            result_list.append((phase_full_info[c],i,a))
        except ValueError as e:
            f_count += 1
            #print(i)
            #print(e)
            #break
        except Exception as e:
            fail_count += 1

            

    print("F errors: ",f_count)
    print("Other errors:", fail_count)
    print("Successes:", len(result_list))
    print("Total compounds", count)
    end = time.process_time()
    end_2 = time.time()
    
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")
        
    return result_list

In [ ]:
from tqdm import tqdm
import time  # Ensure this is imported if not already

def energy_above_hull_4(file_list, thermo_docs):
    start = time.process_time()
    start_2 = time.time()
    
    count = 0
    f_count = 0
    fail_count = 0
    result_list = []
    
    # Using tqdm to track progress
    for c, i in tqdm(enumerate(file_list), total=len(file_list), desc="Processing files"):
        try:
            count += 1
            atoms = read(f"{phase}/{i}")  # CHANGE TO APPROPRIATE FOLDER NAME
            compound_symbol = atoms.symbols
            individual_symbol = atoms.get_chemical_symbols()
            free_energy = atoms.get_potential_energy()
    
            # Energy above hull data
            chemsys = {individual_symbol[0], individual_symbol[2], individual_symbol[3]}
            
            entries = []
            for doc in thermo_docs:
                elements = {e.name for e in doc[0].elements}
                if elements.issubset(chemsys):
                    entry = PDEntry(
                        doc[0],
                        doc[1] * doc[0].num_atoms
                    )
                    entries.append(entry)
    
            pd = PhaseDiagram(entries)
    
            entry = ComputedEntry(
                f"{compound_symbol}",
                free_energy,
                parameters={"run_type": "GGA+U"}
            )
            
            corrected_entry = mpc.process_entry(entry)
            a = pd.get_e_above_hull(corrected_entry)
            result_list.append((phase_full_info[c], i, a))
        
        except ValueError:
            f_count += 1
        except Exception as e:
            fail_count += 1
            print(f"Error in file: {i}, Exception: {e}")
            break  # You may want to remove break if you want to continue processing other files

    print("\nSummary:")
    print(f"F errors: {f_count}")
    print(f"Other errors: {fail_count}")
    print(f"Successes: {len(result_list)}")
    print(f"Total compounds processed: {count}")
    
    end = time.process_time()
    end_2 = time.time()
    
    print(f"CPU Time: {end - start:.2f} seconds")
    print(f"Elapsed Time: {end_2 - start_2:.2f} seconds")
    
    return result_list


In [ ]:
result = energy_above_hull_4(file_list[:],thermo_docs)

In [ ]:
def flatten_once(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(item)  # Unpack one level
        else:
            flattened.append(item)  # Keep non-list items
    return flattened




In [ ]:
flat_list = []
for i in result:
    flat_list.append(flatten_once(i))


In [ ]:
# COMPOUNDS WITH AN ENERGY ABOVE HULL OF LESS THAN 0.05 eV
metastable_phase = []
for i in flat_list:
    if i[-1] <= 0.05:
        metastable_phase.append(i)

print(len(metastable_phase))



In [ ]:
file_path = f'{phase} metastable compounds v2.csv'  # CHANGE FILE NAME

# Writing to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Index","Element Combination", "Oxidation States","Radius of A","Radius of B","Radius of C","Tolerance Factor",
                    "Octahedral factor","File name","Energy above hull"])   
    # Write each row (tuple in this case) to the CSV
    for row in metastable_phase:   
        writer.writerow(row)

print(f"Data saved to {file_path}")



In [ ]:
file_path = f"{phase} full result v2.csv"  # CHANGE FILE NAME

# Writing to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Index","Element Combination", "Oxidation States","Radius of A","Radius of B","Radius of C","Tolerance Factor",
                    "Octahedral factor","File name","Energy above hull"])   
    # Write each row (tuple in this case) to the CSV
    for row in flat_list:   
        writer.writerow(row)

print(f"Data saved to {file_path}")

